# HW4P2: Attention-based Speech Recognition

Welcome to the final assignment in 11785. In this HW, you will work on building a speech recognition system with attention. <br> <br>

HW Writeup: https://piazza.com/class_profile/get_resource/l37uyxe87cq5xn/lam1lcjjj0314e <br>
Kaggle competition link: https://www.kaggle.com/competitions/11-785-f22-hw4p2/ <br>
LAS Paper: https://arxiv.org/pdf/1508.01211.pdf <br>
Attention is all you need:https://arxiv.org/pdf/1706.03762.pdf

# Initial Set-up

In [7]:
!nvidia-smi

Thu Jan  5 16:09:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:21:00.0 Off |                  N/A |
| 62%   80C    P2   268W / 280W |   9449MiB / 24576MiB |     91%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:4B:00.0 Off |                  N/A |
| 41%   

In [8]:
# Install some required libraries
# Feel free to add more if you want
# !pip install -q python-levenshtein torchsummaryX wandb

# Import

In [9]:
import os
import pandas as pd
import numpy as np
import Levenshtein

import torch
import torchaudio

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import gc
from torchsummaryX import summary
import wandb
from glob import glob

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", DEVICE)

Device:  cuda


In [10]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch.nn as nn
from torch.utils.data import DataLoader

In [11]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config

In [12]:
# Global config dict. Feel free to add or change if you want.
config = {
    'exp_name': 'early_submission',
    'batch_size': 96,
    'epochs': 30,
    'lr': 1e-3,
    'weight_decay':  5e-6,
    'lr_scheduler': 'stepLR',
    'step_size': 5,
    'locked_dropout' : 0.15
}

# Toy Data Setup

The toy dataset is very essential for you in this HW. The model which you will be building is complicated and you first need to make sure that it runs on the toy dataset. <br>
In other words, you need convergence - the attention diagonal. Take a look at the write-up for this. <br>
We have given you the following code to download the toy data and load it. You can use it the way it is. But be careful, the transcripts are different from the original data from kaggle. The toy dataset has phonemes but the actual data has characters.

In [13]:
# !wget -q https://cmu.box.com/shared/static/wok08c2z2dp4clufhy79c5ee6jx3pyj9 --content-disposition --show-progress
# !wget -q https://cmu.box.com/shared/static/zctr6mvh7npfn01forli8n45duhp2g85 --content-disposition --show-progress
# !wget -q https://cmu.box.com/shared/static/m2oaek69145ljeu6srtbbb7k0ip6yfup --content-disposition --show-progress
# !wget -q https://cmu.box.com/shared/static/owrjy0tqra3v7zq2ru7mocy2djskydy9 --content-disposition --show-progress

In [14]:
# # Load the toy dataset
# X_train = np.load("f0176_mfccs_train.npy")
# X_valid = np.load("f0176_mfccs_dev.npy")
# Y_train = np.load("f0176_hw3p2_train.npy")
# Y_valid = np.load("f0176_hw3p2_dev.npy")

# # This is how you actually need to find out the different trancripts in a dataset. 
# # Can you think whats going on here? Why are we using a np.unique?
# VOCAB_MAP           = dict(zip(np.unique(Y_valid), range(len(np.unique(Y_valid))))) 
# VOCAB_MAP["[PAD]"]  = len(VOCAB_MAP)
# VOCAB               = list(VOCAB_MAP.keys())

# SOS_TOKEN = VOCAB_MAP["[SOS]"]
# EOS_TOKEN = VOCAB_MAP["[EOS]"]
# PAD_TOKEN = VOCAB_MAP["[PAD]"]

# Y_train = [np.array([VOCAB_MAP[p] for p in seq]) for seq in Y_train]
# Y_valid = [np.array([VOCAB_MAP[p] for p in seq]) for seq in Y_valid]

In [15]:
# # Dataset class for the Toy dataset
# class ToyDataset(torch.utils.data.Dataset):

#     def __init__(self, partition):

#         if partition == "train":
#             self.mfccs = X_train[:, :, :15]
#             self.transcripts = Y_train

#         elif partition == "valid":
#             self.mfccs = X_valid[:, :, :15]
#             self.transcripts = Y_valid

#         assert len(self.mfccs) == len(self.transcripts)

#         self.length = len(self.mfccs)

#     def __len__(self):

#         return self.length

#     def __getitem__(self, i):

#         x = torch.tensor(self.mfccs[i])
#         y = torch.tensor(self.transcripts[i])

#         return x, y

#     def collate_fn(self, batch):

#         x_batch, y_batch = list(zip(*batch))

#         x_lens      = [x.shape[0] for x in x_batch] 
#         y_lens      = [y.shape[0] for y in y_batch] 

#         x_batch_pad = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value= EOS_TOKEN)
#         y_batch_pad = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value= EOS_TOKEN) 
        
#         return x_batch_pad, y_batch_pad, torch.tensor(x_lens), torch.tensor(y_lens)

In [16]:
# train_data = ToyDataset(partition='train') #TODO
# val_data = ToyDataset(partition='valid') # TODO : You can either use the same class with some modifications or make a new one :)

# toy_train_loader =  DataLoader(train_data, shuffle=True, batch_size=config['batch_size'], collate_fn=train_data.collate_fn)
# toy_val_loader =  DataLoader(val_data, shuffle=True, batch_size=config['batch_size'], collate_fn=train_data.collate_fn)

In [17]:
# for data in toy_val_loader:
#     x, y, lx, ly = data
#     print(x.shape, y.shape, lx.shape, ly.shape)
#     break 

# Kaggle Data

In [18]:
# # TODO: Use the same Kaggle code from HW1P2, HW2P2, HW3P2
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# !mkdir /root/.kaggle/

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('') # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

In [19]:
# Download the data
# !kaggle competitions download -c 11-785-f22-hw4p2
# !mkdir 'data'

# !unzip -qo '11-785-f22-hw4p2.zip' -d 'data'

# Dataset Class

In [20]:
# These are the various characters in the transcripts of the datasetW
VOCAB = ['<sos>',   
         'A',   'B',    'C',    'D',    
         'E',   'F',    'G',    'H',    
         'I',   'J',    'K',    'L',       
         'M',   'N',    'O',    'P',    
         'Q',   'R',    'S',    'T', 
         'U',   'V',    'W',    'X', 
         'Y',   'Z',    "'",    ' ', 
         '<eos>']

VOCAB_MAP = {VOCAB[i]:i for i in range(0, len(VOCAB))}

SOS_TOKEN = VOCAB_MAP["<sos>"]
EOS_TOKEN = VOCAB_MAP["<eos>"]

# Dataset and Dataloaders

In [21]:
from dataloader import AudioDataset, AudioDatasetTest
from torch.utils.data import DataLoader

In [22]:
# TODO: Create the datasets and dataloaders
# All these things are similar to HW3P2
# You can reuse the same code

# Create objects for the dataset class
train_data = AudioDataset(VOCAB, VOCAB_MAP, partition='train') #TODO
val_data = AudioDataset(VOCAB, VOCAB_MAP, partition='val') # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest() #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = DataLoader(train_data, shuffle=True, batch_size=config['batch_size'], collate_fn=train_data.collate_fn)#TODO
val_loader = DataLoader(val_data, shuffle=True, batch_size=config['batch_size'], collate_fn=val_data.collate_fn) #TODO
test_loader = DataLoader(test_data, shuffle=False, batch_size=config['batch_size'], collate_fn=test_data.collate_fn) #TODO

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

# The sanity check for shapes also are similar
# Please remember that the only change in the dataset for this HW is the transcripts
# So you are expected to get similar shapes like HW3P2 (Pad, pack and Oh my!)

Batch size:  96
Train dataset samples = 28539, batches = 298
Val dataset samples = 2703, batches = 29
Test dataset samples = 2620, batches = 28


In [23]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([96, 1738, 15]) torch.Size([96, 322]) torch.Size([96]) torch.Size([96])


# Model

In this section you will be building the LAS model from scratch. Before starting to code, please read the writeup, paper and understand the following parts completely.<br>
- Pyramidal Bi-LSTM 
- Listener
- Attention
- Speller

After getting a good grasp of the workings of these modules, start coding. Follow the TODOs carefully. We will also be adding some extra features to the attention mechanism like keys and values which are not originally present in LAS. So we will be creating a hybrid network based on LAS and Attention is All You Need.


## Encoder

In [24]:
class LockedDropout(nn.Module):
    """ LockedDropout applies the same dropout mask to every time step.

    Args:
        p (float): Probability of an element in the dropout mask to be zeroed.
    """

    def __init__(self, p=0.5):
        self.p = p
        super().__init__()

    def forward(self, x):
        """
        Args:
            x (:class:`torch.FloatTensor` [sequence length, batch size, rnn hidden size]): Input to
                apply dropout too.
        """
        if not self.training or not self.p:
            return x
        x = x.clone()
        mask = x.new_empty(1, x.size(1), x.size(2), requires_grad=False).bernoulli_(1 - self.p)
        mask = mask.div_(1 - self.p)
        mask = mask.expand_as(x)
        return x * mask


    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'p=' + str(self.p) + ')'

### Pyramidal Bi-LSTM

In [25]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''
    
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        
        # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size
        self.blstm = torch.nn.LSTM(input_size=4*input_size, hidden_size=hidden_size, num_layers=1, bidirectional=True, batch_first=True)
        self.downsample_factor = 2

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence
        
        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # TODO: Pack Padded Sequence. What output(s) would you get?
        # TODO: Pass the sequence through bLSTM

        # What do you return?
        x_unpacked, len_unpacked = pad_packed_sequence(x_packed, batch_first=True)
        x_downsampled, len_downsampled = self.trunc_reshape(x_unpacked, len_unpacked)
        x_packed = pack_padded_sequence(x_downsampled, len_downsampled, batch_first=True, enforce_sorted=False)
        output, (hn, cn) = self.blstm(x_packed)

        return output

    def trunc_reshape(self, x, x_lens): 
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor
        max_len = x.shape[1]
        if max_len % 2 == 1:
            x = x[:, :-1]
            max_len -= 1
            
        k = self.downsample_factor
        x = [x[:, i:i+k].flatten(start_dim=1) for i in range(0, max_len, 2)]
        x = torch.stack(x, dim=1)
        # x_lens = x_lens // k
        x_lens = torch.div(x_lens, k, rounding_mode='trunc')
        return x, x_lens

### Listener

In [26]:
class Listener(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Listener, self).__init__()

        # The first LSTM at the very bottom
        self.base_lstm1 = torch.nn.LSTM(input_size, encoder_hidden_size, num_layers=1, bidirectional=True, batch_first=True, dropout=0.15)#TODO: Fill this up
        self.base_lstm2 = torch.nn.LSTM(encoder_hidden_size*2, encoder_hidden_size, num_layers=1, bidirectional=True, batch_first=True)#TODO: Fill this up


        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be? 
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # ...
            # ...
            pBLSTM(encoder_hidden_size, 2*encoder_hidden_size),
            pBLSTM(2*encoder_hidden_size, 4*encoder_hidden_size),
            pBLSTM(4*encoder_hidden_size, 8*encoder_hidden_size)
        )

        self.locked_dropout = LockedDropout(p=config['locked_dropout'])
         
    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        
        # TODO: Pack Padded Sequence
        # TODO: Pass it through the first LSTM layer (no truncation)
        # TODO: Pad Packed Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer

        # Remember the number of output(s) each function returns
        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        out, (hn, cn) = self.base_lstm1(x_packed)
        x, x_lens = pad_packed_sequence(out, batch_first=True)
        x = self.locked_dropout(x)

        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        out, (hn, cn) = self.base_lstm2(x_packed)

        output = self.pBLSTMs(out)
        encoder_outputs, encoder_lens = pad_packed_sequence(output, batch_first=True)

        return encoder_outputs, encoder_lens

In [27]:
encoder = Listener(input_size=15, encoder_hidden_size=64).to(DEVICE) # TODO: Initialize Listener
print(encoder)
example_batch = next(iter(train_loader))
print(example_batch[0].shape, example_batch[1].shape, example_batch[2].shape, example_batch[3].shape)
summary(encoder, example_batch[0].to(DEVICE), example_batch[3])
del encoder

/home/jiin/anaconda3/envs/torch/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.15 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Listener(
  (base_lstm1): LSTM(15, 64, batch_first=True, dropout=0.15, bidirectional=True)
  (base_lstm2): LSTM(128, 64, batch_first=True, bidirectional=True)
  (pBLSTMs): Sequential(
    (0): pBLSTM(
      (blstm): LSTM(256, 128, batch_first=True, bidirectional=True)
    )
    (1): pBLSTM(
      (blstm): LSTM(512, 256, batch_first=True, bidirectional=True)
    )
    (2): pBLSTM(
      (blstm): LSTM(1024, 512, batch_first=True, bidirectional=True)
    )
  )
  (locked_dropout): LockedDropout(p=0.15)
)
torch.Size([96, 1670, 15]) torch.Size([96, 291]) torch.Size([96]) torch.Size([96])
                       Kernel Shape    Output Shape     Params  Mult-Adds
Layer                                                                    
0_base_lstm1                      -    [17964, 128]    41.472k    40.448k
1_locked_dropout                  -  [96, 291, 128]          -          -
2_base_lstm2                      -    [17964, 128]    99.328k    98.304k
3_pBLSTMs.0.LSTM_blstm            -     [

/home/jiin/anaconda3/envs/torch/lib/python3.9/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


## Attention (Attend)

### Different ways to compute Attention

1. Dot-product attention
    * raw_weights = bmm(key, query) 
    * Optional: Scaled dot-product by normalizing with sqrt key dimension 
    * Check "Attention is All You Need" Section 3.2.1
    * 1st way is what most TAs are comfortable with, but if you want to explore, check out other methods below


2. Cosine attention
    * raw_weights = cosine(query, key) # almost the same as dot-product xD 

3. Bi-linear attention
    * W = Linear transformation (learnable parameter): d_k -> d_q
    * raw_weights = bmm(key @ W, query)

4. Multi-layer perceptron
    * Check "Neural Machine Translation and Sequence-to-sequence Models: A Tutorial" Section 8.4

5. Multi-Head Attention
    * Check "Attention is All You Need" Section 3.2.2
    * h = Number of heads
    * W_Q, W_K, W_V: Weight matrix for Q, K, V (h of them in total)
    * W_O: d_v -> d_v
    * Reshape K: (B, T, d_k) to (B, T, h, d_k // h) and transpose to (B, h, T, d_k // h)
    * Reshape V: (B, T, d_v) to (B, T, h, d_v // h) and transpose to (B, h, T, d_v // h)
    * Reshape Q: (B, d_q) to (B, h, d_q // h) `
    * raw_weights = Q @ K^T
    * masked_raw_weights = mask(raw_weights)
    * attention = softmax(masked_raw_weights)
    * multi_head = attention @ V
    * multi_head = multi_head reshaped to (B, d_v)
    * context = multi_head @ W_O

In [28]:
def plot_attention(attention): 
    # Function for plotting attention
    # You need to get a diagonal plot
    plt.clf()
    sns.heatmap(attention, cmap='GnBu')
    plt.show()

class Attention(torch.nn.Module):
    '''
    Attention is calculated using the key, value (from encoder hidden states) and query from decoder.
    Here are different ways to compute attention and context:

    After obtaining the raw weights, compute and return attention weights and context as follows.:

    masked_raw_weights  = mask(raw_weights) # mask out padded elements with big negative number (e.g. -1e9 or -inf in FP16)
    attention           = softmax(masked_raw_weights)
    context             = bmm(attention, value)
    
    At the end, you can pass context through a linear layer too.

    '''
    
    def __init__(self, encoder_hidden_size, decoder_output_size, projection_size):
        super(Attention, self).__init__()

        self.key_projection     = nn.Linear(encoder_hidden_size*4, projection_size) # TODO: Define an nn.Linear layer which projects the encoder_hidden_state to keys
        self.value_projection   = nn.Linear(encoder_hidden_size*4, projection_size) # TODO: Define an nn.Linear layer which projects the encoder_hidden_state to value
        self.query_projection   = nn.Linear(decoder_output_size, projection_size) # TODO: Define an nn.Linear layer which projects the decoder_output_state to query
        # Optional : Define an nn.Linear layer which projects the context vector

        self.softmax            = nn.Softmax(dim=1) # TODO: Define a softmax layer. Think about the dimension which you need to apply 
        # Tip: What is the shape of energy? And what are those?
        ## softmax dimension check!!!!!!!!!!!!!

    # As you know, in the attention mechanism, the key, value and mask are calculated only once.
    # This function is used to calculate them and set them to self
    def set_key_value_mask(self, encoder_outputs, encoder_lens):
    
        _, encoder_max_seq_len, _ = encoder_outputs.shape

        self.key      = self.key_projection(encoder_outputs) # TODO: Project encoder_outputs using key_projection to get keys
        self.value    = self.value_projection(encoder_outputs) # TODO: Project encoder_outputs using value_projection to get values

        # encoder_max_seq_len is of shape (batch_size, ) which consists of the lengths encoder output sequences in that batch
        # The raw_weights are of shape (batch_size, timesteps)

        # TODO: To remove the influence of padding in the raw_weights, we want to create a boolean mask of shape (batch_size, timesteps) 
        # The mask is False for all indicies before padding begins, True for all indices after.
        self.padding_mask     = torch.arange(encoder_max_seq_len).unsqueeze(0) >= encoder_lens.unsqueeze(1) # TODO: You want to use a comparison between encoder_max_seq_len and encoder_lens to create this mask. 
        self.padding_mask = self.padding_mask.to(DEVICE)
        # (Hint: Broadcasting gives you a one liner)
        
    def forward(self, decoder_output_embedding):
        # key   : (batch_size, timesteps, projection_size)
        # value : (batch_size, timesteps, projection_size)
        # query : (batch_size, projection_size)

        self.query         = self.query_projection(decoder_output_embedding) # TODO: Project the query using query_projection

        # Hint: Take a look at torch.bmm for the products below 

        query_len = self.query.shape[1] ## projection_size
        raw_weights        = (1/np.sqrt(query_len)) * torch.bmm(self.key, self.query.unsqueeze(-1)).squeeze(-1) # TODO: Calculate raw_weights which is the product of query and key, and is of shape (batch_size, timesteps)
        masked_raw_weights = raw_weights.masked_fill_(self.padding_mask.bool(), -float('inf')) # TODO: Mask the raw_weights with self.padding_mask. 
        # Take a look at pytorch's masked_fill_ function (You want the fill value to be a big negative number for the softmax to make it close to 0)

        attention_weights  = self.softmax(masked_raw_weights) # TODO: Calculate the attention weights, which is the softmax of raw_weights
        # attention_weights : (batch_size, timesteps)
        context            = torch.bmm(attention_weights.unsqueeze(1), self.value).squeeze(1) # TODO: Calculate the context - it is a product between attention_weights and value

        # Hint: You might need to use squeeze/unsqueeze to make sure that your operations work with bmm

        return context, attention_weights # Return the context, attention_weights

## Decoder

### Speller

In [29]:
class Speller(torch.nn.Module):

    def __init__(self, embed_size, decoder_hidden_size, decoder_output_size, vocab_size, attention_module= None):
        super().__init__()

        self.vocab_size         = vocab_size

        self.embedding          = nn.Embedding(self.vocab_size, embed_size, padding_idx=EOS_TOKEN) # TODO: Initialize the Embedding Layer (Use the nn.Embedding Layer from torch), make sure you set the correct padding_idx  

        self.lstm_cells         = torch.nn.Sequential(
                                # Create Two LSTM Cells as per LAS Architecture
                                # What should the input_size of the first LSTM Cell? 
                                # Hint: It takes in a combination of the character embedding and context from attention
                                nn.LSTMCell(embed_size + attention_module.value_projection.weight.shape[0], decoder_hidden_size),
                                nn.LSTMCell(decoder_hidden_size, decoder_output_size)
                                )
    
                                # We are using LSTMCells because process individual time steps inputs and not the whole sequence.
                                # Think why we need this in terms of the query
        self.decoder_output_size = decoder_output_size

        self.char_prob          = nn.Linear(decoder_output_size, vocab_size) # TODO: Initialize the classification layer to generate your probability distribution over all characters

        self.char_prob.weight   = self.embedding.weight # Weight tying

        self.attention          = attention_module

    
    def forward(self, encoder_outputs, encoder_lens, y = None, tf_rate = 1): 

        '''
        Args: 
            embedding: Attention embeddings 
            hidden_list: List of Hidden States for the LSTM Cells
        ''' 

        batch_size, encoder_max_seq_len, _ = encoder_outputs.shape

        if self.training:
            timesteps     = y.shape[1] # The number of timesteps is the sequence of length of your transcript during training
            label_embed   = self.embedding(y) # Embeddings of the transcript, when we want to use teacher forcing
        else:
            timesteps     = 600 # 600 is a design choice that we recommend, however you are free to experiment.
        
        # INITS
        predictions     = []

        # Initialize the first character input to your decoder, SOS
        char            = torch.full((batch_size,), fill_value=SOS_TOKEN, dtype= torch.long).to(DEVICE) 

        # Initialize a list to keep track of LSTM Cell Hidden and Cell Memory States, to None
        hidden_states   = [None]*len(self.lstm_cells) 

        attention_plot          = []
        context                 = torch.zeros(batch_size, self.decoder_output_size).to(DEVICE) # TODO: Initialize context (You have a few choices, refer to the writeup )
        attention_weights       = torch.zeros(batch_size, encoder_max_seq_len) # Attention Weights are zero if not using Attend Module

        # Set Attention Key, Value, Padding Mask just once
        if self.attention != None:
            self.attention.set_key_value_mask(encoder_outputs, encoder_lens)


        for t in range(timesteps):
            char_embed = self.embedding(char) #TODO: Generate the embedding for the character at timestep t

            if self.training and t > 0:
                # TODO: We want to decide which embedding to use as input for the decoder during training
                # We can use the embedding of the transcript character or the embedding of decoded/predicted character, from the previous timestep 
                # Using the embedding of the transcript character is teacher forcing, it is very important for faster convergence
                # Use a comparison between a random probability and your teacher forcing rate, to decide which embedding to use

                char_embed = label_embed[:, t-1, :] # TODO
      
            decoder_input_embedding = torch.cat([char_embed, context], dim=1) # TODO: What do we want to concatenate as input to the decoder? (Use torch.cat)
            # char_embed : (batch_size, embed_size)
            # context : (batch_size, projection_size)
            
            # Loop over your lstm cells
            # Each lstm cell takes in an embedding 
            for i in range(len(self.lstm_cells)):
                # An LSTM Cell returns (h,c) -> h = hidden state, c = cell memory state
                # Using 2 LSTM Cells is akin to a 2 layer LSTM looped through t timesteps 
                # The second LSTM Cell takes in the output hidden state of the first LSTM Cell (from the current timestep) as Input, along with the hidden and cell states of the cell from the previous timestep
                hidden_states[i] = self.lstm_cells[i](decoder_input_embedding, hidden_states[i]) 
                decoder_input_embedding = hidden_states[i][0]

            # The output embedding from the decoder is the hidden state of the last LSTM Cell
            decoder_output_embedding = hidden_states[-1][0] ## s(t) : (batch_size, decoder_output_size)

            # We compute attention from the output of the last LSTM Cell
            if self.attention != None:
                context, attention_weights = self.attention(decoder_output_embedding) # The returned query is the projected query

            # context : (batch_size, projection_size)
            # attention weights : (batch_size, timesteps)
            attention_plot.append(attention_weights[0].detach().cpu())

            output_embedding     = torch.cat([decoder_output_embedding, context], dim=1) # TODO: Concatenate the projected query with context for the output embedding
            # Hint: How can you get the projected query from attention
            # If you are not using attention, what will you use instead of query?

            char_prob            = self.char_prob(output_embedding)
            
            # Append the character probability distribution to the list of predictions 
            predictions.append(char_prob)

            char = torch.argmax(char_prob, dim=1) # TODO: Get the predicted character for the next timestep from the probability distribution 
            # (Hint: Use Greedy Decoding for starters)

        attention_plot  = torch.stack(attention_plot) # TODO: Stack list of attetion_plots 
        predictions     = torch.stack(predictions) # TODO: Stack list of predictions 

        return predictions, attention_plot

## Sequence-to-Sequence Model

### LAS

In [30]:
class LAS(torch.nn.Module):
    def __init__(self, input_size, encoder_hidden_size, 
                 vocab_size, embed_size,
                 decoder_hidden_size, decoder_output_size,
                 projection_size= 128):
        
        super(LAS, self).__init__()

        self.encoder        = Listener(input_size=15, encoder_hidden_size=64) # TODO: Initialize Encoder
        attention_module    = Attention(encoder_hidden_size, decoder_output_size, projection_size) # TODO: Initialize Attention
        self.decoder        = Speller(embed_size, decoder_hidden_size, decoder_output_size, vocab_size, attention_module) # TODO: Initialize Decoder, make sure you pass the attention module 

    def forward(self, x, x_lens, y = None, tf_rate = 1):

        encoder_outputs, encoder_lens = self.encoder(x, x_lens) # from Listener
        predictions, attention_plot = self.decoder(encoder_outputs, encoder_lens, y, tf_rate)
        predictions = predictions.permute(1,0,2)
        
        return predictions, attention_plot

# Training Setup

## Model Setup










In [31]:
# Baseline LAS has the following configuration:
# Encoder bLSTM/pbLSTM Hidden Dimension of 512 (256 per direction)
# Decoder Embedding Layer Dimension of 256
# Decoder Hidden Dimension of 512 
# Decoder Output Dimension of 128
# Attention Projection Size of 128
# Feel Free to Experiment with this 

model = LAS(
    input_size=15, encoder_hidden_size=256, 
    vocab_size=len(VOCAB), embed_size=256,
    decoder_hidden_size=512, decoder_output_size=128,
    projection_size= 128
    # Initialize your model 
    # Read the paper and think about what dimensions should be used
    # You can experiment on these as well, but they are not requried for the early submission
    # Remember that if you are using weight tying, some sizes need to be the same
)

model = model.to(DEVICE)
# print(model)

summary(model, 
        x= example_batch[0].to(DEVICE), 
        x_lens= example_batch[3], 
        y= example_batch[1].to(DEVICE))

/home/jiin/anaconda3/envs/torch/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.15 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/home/jiin/anaconda3/envs/torch/lib/python3.9/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


                                               Kernel Shape    Output Shape  \
Layer                                                                         
0_encoder.LSTM_base_lstm1                                 -    [17964, 128]   
1_encoder.LockedDropout_locked_dropout                    -  [96, 291, 128]   
2_encoder.LSTM_base_lstm2                                 -    [17964, 128]   
3_encoder.pBLSTMs.0.LSTM_blstm                            -     [8954, 256]   
4_encoder.pBLSTMs.1.LSTM_blstm                            -     [4449, 512]   
...                                                     ...             ...   
1750_decoder.lstm_cells.LSTMCell_0                        -       [96, 512]   
1751_decoder.lstm_cells.LSTMCell_1                        -       [96, 128]   
1752_decoder.attention.Linear_query_projection   [128, 128]       [96, 128]   
1753_decoder.attention.Softmax_softmax                    -        [96, 36]   
1754_decoder.Linear_char_prob                     [2

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder.LSTM_base_lstm1,-,"[17964, 128]",41472.0,40448.0
1_encoder.LockedDropout_locked_dropout,-,"[96, 291, 128]",NaN,NaN
2_encoder.LSTM_base_lstm2,-,"[17964, 128]",99328.0,98304.0
3_encoder.pBLSTMs.0.LSTM_blstm,-,"[8954, 256]",395264.0,393216.0
4_encoder.pBLSTMs.1.LSTM_blstm,-,"[4449, 512]",1576960.0,1572864.0
...,...,...,...,...
1750_decoder.lstm_cells.LSTMCell_0,-,"[96, 512]",NaN,1835008.0
1751_decoder.lstm_cells.LSTMCell_1,-,"[96, 128]",NaN,327680.0
1752_decoder.attention.Linear_query_projection,"[128, 128]","[96, 128]",NaN,16384.0


## Optimizer, Scheduler, Loss

In [32]:
optimizer   = torch.optim.Adam(model.parameters(), lr= config['lr'], amsgrad= True, weight_decay=config['weight_decay'])
criterion   = torch.nn.CrossEntropyLoss(reduction='none') # Why are we using reduction = 'none' ? 
scaler      = torch.cuda.amp.GradScaler()
scheduler   = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=0.8)

# Optional: Create a custom class for a Teacher Force Schedule 

# Levenshtein Distance

In [33]:
# We have given you this utility function which takes a sequence of indices and converts them to a list of characters
def indices_to_chars(indices, vocab):
    tokens = []
    for i in indices: # This loops through all the indices
        if int(i) == SOS_TOKEN: # If SOS is encountered, dont add it to the final list
            continue
        elif int(i) == EOS_TOKEN: # If EOS is encountered, stop the decoding process
            break
        else:
            tokens.append(vocab[i])
    return tokens

# To make your life more easier, we have given the Levenshtein distantce / Edit distance calculation code
def calc_edit_distance(predictions, y, ly, vocab= VOCAB, print_example= False):

    dist                = 0
    batch_size, seq_len = predictions.shape

    for batch_idx in range(batch_size): 

        y_sliced    = indices_to_chars(y[batch_idx,0:ly[batch_idx]], vocab)
        pred_sliced = indices_to_chars(predictions[batch_idx], vocab)

        # Strings - When you are using characters from the AudioDataset
        y_string    = ''.join(y_sliced)
        pred_string = ''.join(pred_sliced)
        
        dist        += Levenshtein.distance(pred_string, y_string)
        # Comment the above abd uncomment below for toy dataset 
        # dist      += Levenshtein.distance(y_sliced, pred_sliced)

    if print_example: 
        # Print y_sliced and pred_sliced if you are using the toy dataset
        print("Ground Truth : ", y_string)
        print("Prediction   : ", pred_string)
        # print("Ground Truth : ", y_sliced)
        # print("Prediction   : ", pred_sliced)
        
    dist/=batch_size
    return dist

# Train and Evaluate

In [34]:
def train(model, dataloader, criterion, optimizer, teacher_forcing_rate):

    model.train()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    running_loss        = 0.0
    running_perplexity  = 0.0
    
    for i, (x, y, lx, ly) in enumerate(dataloader):

        optimizer.zero_grad()

        x, y, lx, ly = x.to(DEVICE), y.to(DEVICE), lx, ly

        with torch.cuda.amp.autocast():

            predictions, attention_plot = model(x, lx, y= y, tf_rate= teacher_forcing_rate)

            # Predictions are of Shape (batch_size, timesteps, vocab_size). 
            # Transcripts are of shape (batch_size, timesteps) Which means that you have batch_size amount of batches with timestep number of tokens.
            # So in total, you have batch_size*timesteps amount of characters.
            # Similarly, in predictions, you have batch_size*timesteps amount of probability distributions.
            # How do you need to modify transcipts and predictions so that you can calculate the CrossEntropyLoss? Hint: Use Reshape/View and read the docs
            loss        = criterion(predictions.permute(0,2,1), y) # TODO: Cross Entropy Loss

            # print(loss.shape, ly.shape)  #(batch, max_len) (batch)
            mask        = torch.ones(loss.shape).to(DEVICE) # TODO: Create a boolean mask using the lengths of your transcript that remove the influence of padding indices (in transcripts) in the loss 
            for b in range(loss.shape[0]):
                mask[b][ly[b]:] = 0
            masked_loss = (mask * loss).mean() # Product between the mask and the loss, divided by the mask's sum. Hint: You may want to reshape the mask too 
            perplexity  = torch.exp(masked_loss) # Perplexity is defined the exponential of the loss

            running_loss        += masked_loss.item()
            running_perplexity  += perplexity.item()
        
        # Backward on the masked loss
        scaler.scale(masked_loss).backward()

        # Optional: Use torch.nn.utils.clip_grad_norm to clip gradients to prevent them from exploding, if necessary
        # If using with mixed precision, unscale the Optimizer First before doing gradient clipping
        
        scaler.step(optimizer)
        scaler.update()
        

        batch_bar.set_postfix(
            loss="{:.04f}".format(running_loss/(i+1)),
            perplexity="{:.04f}".format(running_perplexity/(i+1)),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])),
            tf_rate='{:.02f}'.format(teacher_forcing_rate))
        batch_bar.update()

        del x, y, lx, ly
        # torch.cuda.empty_cache()


    running_loss /= len(dataloader)
    running_perplexity /= len(dataloader)
    batch_bar.close()

    return running_loss, running_perplexity, attention_plot

In [35]:
def validate(model, dataloader):

    model.eval()

    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc="Val")

    running_lev_dist = 0.0

    for i, (x, y, lx, ly) in enumerate(dataloader):

        x, y, lx, ly = x.to(DEVICE), y.to(DEVICE), lx, ly

        with torch.inference_mode():
            predictions, attentions = model(x, lx, y = None)

        # Greedy Decoding
        greedy_predictions   = torch.argmax(predictions, dim=2) # TODO: How do you get the most likely character from each distribution in the batch?

        # Calculate Levenshtein Distance
        running_lev_dist    += calc_edit_distance(greedy_predictions, y, ly, VOCAB, print_example = False) # You can use print_example = True for one specific index i in your batches if you want

        batch_bar.set_postfix(
            dist="{:.04f}".format(running_lev_dist/(i+1)))
        batch_bar.update()

        del x, y, lx, ly
        torch.cuda.empty_cache()

    batch_bar.close()
    running_lev_dist /= len(dataloader)

    return running_lev_dist#, running_loss, running_perplexity, 

# Wandb

In [36]:
# Login to Wandb
# Initialize your Wandb Run Here
# Optional: Save your model architecture in a txt file, and save the file to Wandb
import wandb
wandb.login(key="db668044fcf11bc7352d5c79ac0deb75ac86a16b")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jiin. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jiin/.netrc


In [ ]:
run = wandb.init(
    name = config['exp_name'], ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw4p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

# Experiments

In [ ]:
best_lev_dist = float("inf")
tf_rate = 1.0

for epoch in range(0, config['epochs']):
    
    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    # Call train and validate 
    train_loss, train_perplexity, attention_plot = train(model, train_loader, criterion, optimizer, tf_rate)
    valid_dist = validate(model, val_loader)
    curr_lr = optimizer.param_groups[0]['lr']

    # Print your metrics
    print('Train loss = {:.4f}'.format(train_loss))
    print('Valid distance = {:.4f}'.format(valid_dist))

    # Plot Attention 
    plot_attention(attention_plot)

    # Log metrics to Wandb
    wandb.log({"train_loss":train_loss, "train_perplexity":train_perplexity, "validation_dist": valid_dist, "learning_rate": curr_lr})

    # Optional: Scheduler Step / Teacher Force Schedule Step
    scheduler.step()
    tf_rate = tf_rate * 0.9


    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        # Save your model checkpoint here
        torch.save({'state_dict':model.state_dict(),
        'optimizer_state':optimizer.state_dict(),
        'epoch': epoch}, 'models/{}_checkpoint.pth'.format(config['exp_name']))


Epoch: 1/30


Train:   0%|          | 0/298 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Testing

In [ ]:
# Optional: Load your best model Checkpoint here
model = LAS(
    input_size=15, encoder_hidden_size=256, 
    vocab_size=len(VOCAB), embed_size=256,
    decoder_hidden_size=512, decoder_output_size=128,
    projection_size= 128
).to(DEVICE)

model.load_state_dict = torch.load('models/{}_checkpoint.pth'.format(config['exp_name'])['state_dict'])

In [ ]:
# TODO: Create a testing function similar to validation 
# TODO: Create a file with all predictions 
# TODO: Submit to Kaggle

In [ ]:
def test(model, dataloader):

    model.eval()

    all_predictions = []


    for i, (x, y, lx, ly) in tqdm(enumerate(dataloader)):

        x, y, lx, ly = x.to(DEVICE), y.to(DEVICE), lx, ly

        with torch.inference_mode():
            predictions, attentions = model(x, lx, y = None)

        # Greedy Decoding
        greedy_predictions   = torch.argmax(predictions, dim=2) # TODO: How do you get the most likely character from each distribution in the batch?

        all_predictions.append(greedy_predictions)

        del x, y, lx, ly

    return all_predictions